## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key



In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Leningradskiy,RU,2022-07-21 17:02:08,69.3833,178.4167,33.48,99,100,12.97
1,1,Mar Del Plata,AR,2022-07-21 16:59:41,-38.0023,-57.5575,56.08,55,0,14.00
2,2,Namibe,AO,2022-07-21 17:02:09,-15.1961,12.1522,64.47,81,0,5.32
3,3,Rikitea,PF,2022-07-21 17:02:09,-23.1203,-134.9692,74.55,73,36,12.15
4,4,Mataura,NZ,2022-07-21 17:02:09,-46.1927,168.8643,38.48,96,100,1.59


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Hidalgotitlan,MX,2022-07-21 17:02:10,17.7833,-94.6333,84.22,69,52,3.85
10,10,Puerto Ayora,EC,2022-07-21 16:59:05,-0.7393,-90.3518,75.16,87,82,8.99
15,15,Sotouboua,TG,2022-07-21 17:02:12,8.5667,0.9833,78.21,80,84,4.00
17,17,Fare,PF,2022-07-21 17:02:12,-16.7000,-151.0167,76.46,75,9,11.07
21,21,Miri,MY,2022-07-21 17:02:14,4.4148,114.0089,79.75,83,95,5.77
33,33,Marzuq,YE,2022-07-21 17:02:22,14.4000,46.4667,78.75,61,4,9.57
34,34,Banda Aceh,ID,2022-07-21 16:59:33,5.5577,95.3222,81.18,75,100,7.16
35,35,Ribeira Grande,PT,2022-07-21 17:02:22,38.5167,-28.7000,75.60,83,40,8.05
38,38,Atuona,PF,2022-07-21 17:02:23,-9.8000,-139.0333,77.16,76,12,15.84
39,39,Itacarambi,BR,2022-07-21 17:02:24,-15.1022,-44.0919,85.66,22,11,7.43


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


Unnamed: 0    189
City          189
Country       189
Date          189
Lat           189
Lng           189
Max Temp      189
Humidity      189
Cloudiness    189
Wind Speed    189
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count().dropna()

Unnamed: 0    189
City          189
Country       189
Date          189
Lat           189
Lng           189
Max Temp      189
Humidity      189
Cloudiness    189
Wind Speed    189
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Hidalgotitlan,MX,84.22,17.7833,-94.6333,
10,Puerto Ayora,EC,75.16,-0.7393,-90.3518,
15,Sotouboua,TG,78.21,8.5667,0.9833,
17,Fare,PF,76.46,-16.7000,-151.0167,
21,Miri,MY,79.75,4.4148,114.0089,
33,Marzuq,YE,78.75,14.4000,46.4667,
34,Banda Aceh,ID,81.18,5.5577,95.3222,
35,Ribeira Grande,PT,75.60,38.5167,-28.7000,
38,Atuona,PF,77.16,-9.8000,-139.0333,
39,Itacarambi,BR,85.66,-15.1022,-44.0919,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,"type": "lodging","key": g_key}


In [10]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Hidalgotitlan,MX,84.22,17.7833,-94.6333,
10,Puerto Ayora,EC,75.16,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Sotouboua,TG,78.21,8.5667,0.9833,Auberge Médina
17,Fare,PF,76.46,-16.7000,-151.0167,Maitai Lapita Village Huahine
21,Miri,MY,79.75,4.4148,114.0089,"Mega Hotel, Miri"
...,...,...,...,...,...,...
552,Surt,LY,81.05,31.2089,16.5887,الهادي ابوشعاله سرت
554,Wichita Falls,US,88.25,33.9137,-98.4934,Quality Inn & Suites Wichita Falls I-44
555,Gwadar,PK,83.16,25.1216,62.3254,Sadaf Resort
557,Bandarbeyla,SO,76.23,9.4942,50.8122,JABIR HOTEL


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count().dropna()

City          189
Country       189
Max Temp      189
Lat           189
Lng           189
Hotel Name    189
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

In [14]:
# 8b. Export the City_Data into a csv
# clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))